In [3]:
#pip install gensim

In [7]:
import gensim
from gensim import corpora
from gensim.models import LdaModel
from gensim.models import CoherenceModel
from gensim.utils import simple_preprocess
import csv
import pandas as pd

In [2]:
def leer_datos_csv(archivo_csv):
    datos = []
    with open(archivo_csv, 'r') as archivo:
        lector_csv = csv.reader(archivo)
        encabezado = next(lector_csv)

        for fila in lector_csv:
            datos.append(fila)

    return datos

In [3]:
nombre_archivo = 'libros.csv'
datos = leer_datos_csv(nombre_archivo)

In [4]:
titulos_final = []

for fila in datos:
    for f in fila:
        titulos = []
        titulos.append(f.split('|')[1])
        break
    titulos_final.append(titulos[0])
titulos_final

['Educacion Matematica',
 'Matematica: Razonamiento Y Aplicaciones 10/e',
 'Gaceta matemática',
 'Matemática elemental',
 'Introducción a la matemática de los juegos',
 'Revista matemática hispano-americana',
 'Didactica De La Matematica En La Educacion Primaria Intercultural Bilingue',
 'Inteligencia Matematica',
 'Matematica Preuniversitaria',
 'Matemática elemental',
 'Cálculo',
 'Calculo Diferencial E Integral',
 'Calculo',
 'Maestría en el cálculo',
 'Calculus. I',
 'Cálculo en variedades',
 'Calculo 4000',
 'Cálculo Integral',
 'Programación y cálculo numérico',
 'Cálculo numérico',
 'Algebra y Trigonometria',
 'Algebra Elemental',
 'Algebra Lineal Y Sus Aplicaciones',
 'algebra',
 'Álgebra',
 'Recent Research on Pure and Applied Algebra',
 'Cuestiones sobre álgebra lineal',
 'Algebra intermedia',
 'Algebra Lineal',
 'Polynomials',
 'Pensamiento Computacional 2°. Básico - IGER',
 'Pensamiento Computacional 4.° Bachilerato',
 'Separata Pensamiento Computacional - Segundo Básico Se

In [6]:
# Preprocesamiento de los titulos
tokenizados = [titulo.lower().split() for titulo in titulos_final]

# dict
diccionario = corpora.Dictionary(tokenizados)

# corpus
corpus = [diccionario.doc2bow(tokens) for tokens in tokenizados]

# modelo LDA
lda_model = LdaModel(corpus=corpus, id2word=diccionario, num_topics=3, passes=10)

# palabras clave para cada tema
topics = lda_model.print_topics(num_words=5)
for topic in topics:
    print(topic)

# coherencia del modelo
coherence_model = CoherenceModel(model=lda_model, texts=tokenizados, dictionary=diccionario, coherence='c_v')
coherence_score = coherence_model.get_coherence()
print("Coherence Score:", coherence_score)
print()
'''Este enfoque busca medir la similitud entre las palabras dentro de un tema y evaluar si tienen una conexión semántica fuerte. 
Cuanto mayor sea el Coherence Score, más coherentes y semánticamente relacionadas se consideran las palabras en un tema.'''

(0, '0.094*"de" + 0.050*"la" + 0.029*"del" + 0.027*"en" + 0.024*"historia"')
(1, '0.037*"cálculo" + 0.032*"pensamiento" + 0.032*"computacional" + 0.028*"algebra" + 0.025*"taylor\'s"')
(2, '0.037*"en" + 0.026*"y" + 0.025*"el" + 0.024*"derivadas" + 0.024*"ecuaciones"')
Coherence Score: 0.5446399261726006



'Este enfoque busca medir la similitud entre las palabras dentro de un tema y evaluar si tienen una conexión semántica fuerte. \nCuanto mayor sea el Coherence Score, más coherentes y semánticamente relacionadas se consideran las palabras en un tema.'

In [31]:
# guardar el modelo
nombre_archivo = 'modelos/modelo_lda'
lda_model.save(nombre_archivo)

In [10]:
predicciones = []

for documento in titulos_final:
    documento_pre = simple_preprocess(documento)
    
    # vect
    documento_vector = diccionario.doc2bow(documento_pre)

    # inferencia del tema
    distribucion_temas = lda_model.get_document_topics(documento_vector)

    # probabilidad más alta
    tema_clasificado = max(distribucion_temas, key=lambda x: x[1])
    numero_tema = tema_clasificado[0]
    probabilidad_tema = tema_clasificado[1]

    # predict a la lista
    predicciones.append((documento, numero_tema, probabilidad_tema))

# df con las predicciones
columnas = ['Texto', 'Tema', 'Probabilidad']
df_predicciones = pd.DataFrame(predicciones, columns=columnas)

In [11]:
df_predicciones.head()

,Texto,Tema,Probabilidad
0,Educacion Matematica,2,0.773307
1,Matematica: Razonamiento Y Aplicaciones 10/e,2,0.829521
2,Gaceta matemática,1,0.776865
3,Matemática elemental,1,0.777377
4,Introducción a la matemática de los juegos,1,0.484927


In [12]:
# guardo el csv
df_predicciones.to_csv('bases/df_predicciones.csv',sep='|')